In [20]:
import requests

res = requests.get('https://api.protonx.io/organizations/64be83b9050dda00146cb4be/learning-paths/published')

In [21]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
from dotenv import load_dotenv
import os
load_dotenv()

GPT_MODEL = "gpt-4o"
client = OpenAI(api_key=os.getenv('open_ai_key'))

In [22]:
# @title Xử lý dữ liệu: Liệt kê danh sách lớp học trên nền tảng


import pandas as pd
# Flatten the JSON structure
flattened_data = []

for item in res.json():
    root_stage = item.pop('rootStage', {})
    child_stages = root_stage.pop('childStages', [])
    for child_stage in child_stages:
        nested_child_stages = child_stage.pop('childStages', [])
        for nested_child_stage in nested_child_stages:
            nested_child_stage['parentStage'] = child_stage['_id']
            nested_child_stage['grandparentStage'] = item['_id']
            flattened_data.append(nested_child_stage)
        child_stage['parentStage'] = item['_id']
        flattened_data.append(child_stage)
    item['rootStage'] = root_stage
    flattened_data.append(item)

# Convert to DataFrame
df = pd.json_normalize(flattened_data)

data = df["courses"].values

# Function to extract all course information
def extract_courses(data):
    courses = []
    for item in data:
        if isinstance(item, list):
            for subitem in item:
                if isinstance(subitem, dict):
                    courses.append(subitem)
    return courses

# Extracting the courses
courses = extract_courses(data)

# Converting to pandas DataFrame
df_courses = pd.DataFrame(courses)

# Displaying the DataFrame
df_courses.head()

,autoApproval,_id,name,descriptionHeading,imageUrl,openingDay
0,False,649901602380fa001955c84a,Kỹ sư dữ liệu (Video),Trở thành kỹ sư dữ liệu có thể xây dựng những ...,https://storage.googleapis.com/mle-courses-pro...,NaN
1,False,682aaf065dfe9600125aa398,MLOps - Triển khai mô hình học máy trên sản ph...,Khóa học MLOps mang đến cho bạn kiến thức và k...,https://storage.googleapis.com/mle-courses-pro...,NaN
2,True,6833f056b735c100199cddfc,Khóa học chuyên crawl dữ liệu,"Khóa học ""Chuyên crawl dữ liệu"" dành cho những...",https://storage.googleapis.com/mle-courses-pro...,NaN
3,False,67bfd366eaa2b50012815681,Nền tảng xử lý ngôn ngữ tự nhiên 06,"Trong khóa học này, học viên sẽ được hướng dẫn...",https://storage.googleapis.com/mle-courses-pro...,NaN
4,True,669f392f9ae80d0019c696ac,Toán cho học sâu,Lớp học bổ trợ cho lớp học AI nền tảng,NaN,NaN


Format tham số mà người dùng muốn hỏi về lớp học
- Ở đây đặt là query. LLMs sẽ quyết định xem là thông tin này có tồn tại trong truy vấn của người dùng hay không.

In [23]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_courses",
            "description": "Use this function to answer user questions about courses.",
            "parameters": {
                "type": "object",
                "properties":{
                    "query":{
                        "type":"string",
                        "description": """Get query from user intent"""
                    },
                },
                "required": ["query"],
            },
        }
    },
    {
      "type": "function",
      "function": {
        "name": "delete_course",
        "description": "Deletes a course identified by course ID that must contain 'protonx'",
        "parameters": {
          "type": "object",
          "required": [
            "courseId"
          ],
          "properties": {
            "courseId": {
              "type": "string",
              "description": "The ID of the course to delete, which must contain the substring 'protonx'"
            }
          },
          "additionalProperties": False
        },
        "strict": True
      }
    }
]

In [24]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [25]:
messages = []
messages.append({"role": "user", "content": "Tìm khóa học agent"})
chat_response = chat_completion_request(
    messages, tools=tools
)

In [26]:
print(chat_response)

ChatCompletion(id='chatcmpl-C43NaSb6GHjrztSILVIpIe86kspmL', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_7QP42Xs5qnkc20uZmJ0HPi7i', function=Function(arguments='{"query":"agent"}', name='search_courses'), type='function')]))], created=1755082338, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_ff25b2783a', usage=CompletionUsage(completion_tokens=14, prompt_tokens=108, total_tokens=122, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [27]:
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_7QP42Xs5qnkc20uZmJ0HPi7i', function=Function(arguments='{"query":"agent"}', name='search_courses'), type='function')])

In [28]:
# Step 2: determine if the response from the model includes a tool call.
tool_calls = assistant_message.tool_calls
if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = eval(tool_calls[0].function.arguments)


In [29]:
tool_function_name = tool_calls[0].function.name
tool_function_name

'search_courses'

In [30]:
tool_query_string

{'query': 'agent'}

In [31]:
tool_calls[0].function.arguments


'{"query":"agent"}'

In [32]:
tool_query_string = eval(tool_calls[0].function.arguments)
tool_query_string

{'query': 'agent'}

In [33]:
def search_courses(query):
    # Logic to search for courses based on the query
    df_courses_filtered = df_courses[df_courses['descriptionHeading'].str.contains(query, case=False)]
    return df_courses_filtered

# search_courses(tool_query_string)

In [34]:
def delete_course(courseId):
    # Delete a course by courseId
    print("Deleting {courseId}".format(courseId=courseId))




In [35]:
if tool_function_name == "delete_course":
    delete_course(tool_query_string["courseId"])
elif tool_function_name == "search_courses":
    print(search_courses(tool_query_string["query"]))

   autoApproval                       _id                           name  \
9          True  67d0ef8beaa2b5001290ff7b  Lớp học nâng cao chuyên Agent   

                                  descriptionHeading  \
9  Khóa học "Lớp học nâng cao chuyên RAG + Agent"...   

                                            imageUrl  openingDay  
9  https://storage.googleapis.com/mle-courses-pro...         NaN  


In [36]:
mapping = {
    "search_courses": search_courses,
    "delete_course": delete_course,
}
mapping[tool_function_name]

<function __main__.search_courses(query)>

## Keyword lấy được được từ người dùng


In [37]:
tool_query_string

{'query': 'agent'}

Sử dụng keywork này để truy xuất vào Pandas Frame

In [38]:
query = "agent"
df_courses[df_courses['descriptionHeading'].str.contains(query, case=False)]

,autoApproval,_id,name,descriptionHeading,imageUrl,openingDay
9,True,67d0ef8beaa2b5001290ff7b,Lớp học nâng cao chuyên Agent,"Khóa học ""Lớp học nâng cao chuyên RAG + Agent""...",https://storage.googleapis.com/mle-courses-pro...,NaN


In [39]:
df_courses['descriptionHeading']

0     Trở thành kỹ sư dữ liệu có thể xây dựng những ...
1     Khóa học MLOps mang đến cho bạn kiến thức và k...
2     Khóa học "Chuyên crawl dữ liệu" dành cho những...
3     Trong khóa học này, học viên sẽ được hướng dẫn...
4                Lớp học bổ trợ cho lớp học AI nền tảng
5     Lớp học Pytorch, xây dựng những ứng dụng AI kh...
6              Đọc và lập trình các nghiên cứu mới nhất
7     Khóa học "Trí tuệ nhân tạo nền tảng - TF10" gi...
8     Khóa học MLOps mang đến cho bạn kiến thức và k...
9     Khóa học "Lớp học nâng cao chuyên RAG + Agent"...
10    Lớp học chuyên lập trình Chatbot phù hợp cho n...
11                Lớp học nền tảng thị giác máy tính 01
12    Khóa học "Trí tuệ nhân tạo nền tảng - TF10" gi...
13               Lớp học bổ trợ cho lớp học AI nền tảng
14             Đọc và lập trình các nghiên cứu mới nhất
15    Khóa học MLOps mang đến cho bạn kiến thức và k...
16    Sử dụng các công cụ như N8N và Make để tự động...
Name: descriptionHeading, dtype: object

In [40]:
search_courses("thị giác máy tính")

,autoApproval,_id,name,descriptionHeading,imageUrl,openingDay
11,False,660137cff83560001912c743,Thị giác máy tính nền tảng,Lớp học nền tảng thị giác máy tính 01,https://storage.googleapis.com/mle-courses-pro...,NaN
